In [2]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DateType
import pyspark.sql.functions as f

In [3]:
#SPARK CONFIG
warehouse_location = 'hdfs://hdfs-nn:9000/demo/'

builder = SparkSession \
    .builder \
    .appName("Python Spark") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
spark.sql(
    """
    DROP TABLE IF EXISTS UC3_LS
    """
)

DataFrame[]

In [4]:
#DROP TABLE
spark.sql(
    """
    DROP TABLE IF EXISTS UC3_LS.stats
    """
)

DataFrame[]

In [5]:
# create gold database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS UC3_LS LOCATION 'hdfs://hdfs-nn:9000/demo/gold/UC3_LS.db/'
    """
)


spark.sql(
    """
    CREATE EXTERNAL TABLE UC3_LS.stats (
    
        Div STRING,
        Team STRING,
        Date STRING,
        Media_golos DOUBLE,
        Media_remates DOUBLE,
        Media_remates_baliza DOUBLE,
        Media_cantos DOUBLE,
        Eficacia DOUBLE
        
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/demo/gold/UC3_LS.db/stats/'
    """
)

DataFrame[]

In [6]:
from pyspark.sql.functions import substring, avg, sum, round
from pyspark.sql.functions import col, concat_ws, concat, lit, desc, asc

# read data from tables in projeto.db
tabela = spark.read.format("delta").load("hdfs://hdfs-nn:9000/demo/silver/league_stats.db/deltalake_table/")


silver_table = tabela \
    .withColumn("Date", concat(lit("20"), substring(col("Date"), 7, 8)))


silver_table.show()

+--------------+-----------+----+----+----+----+---+---+---+---+---+---+-------+
|      HomeTeam|   AwayTeam| FTR|Date|FTHG|FTAG| HS| AS|HST|AST| HC| AC|    Div|
+--------------+-----------+----+----+----+----+---+---+---+---+---+---+-------+
|     Liverpool|    Arsenal|Draw|2014|   2|   2| 27|  7| 10|  3| 10|  6|England|
|       Swansea|Aston Villa|Home|2014|   1|   0|  7| 11|  2|  3|  4|  0|England|
|     Liverpool|  Leicester|Draw|2015|   2|   2| 18| 16|  6|  3|  4|  5|England|
|      West Ham|  Newcastle|Home|2015|   2|   0| 17| 14|  3|  4|  6|  7|England|
|      Man City|  Newcastle|Home|2015|   6|   1| 23|  6| 11|  4|  6|  3|England|
|    Sunderland|Southampton|Away|2015|   0|   1|  9| 11|  3|  4| 10| 11|England|
|   Aston Villa|    Everton|Away|2016|   1|   3| 14| 10|  5|  7|  9|  8|England|
|    Man United|    Watford|Home|2016|   1|   0| 14| 13|  3|  3|  9|  7|England|
|       Norwich|   Man City|Draw|2016|   0|   0|  5| 15|  0|  3|  3|  8|England|
|      Man City|    Watford|

In [7]:
homeTeam_table = silver_table \
    .groupBy("HomeTeam", "Date", "Div") \
    .agg(
        avg(silver_table.FTHG).alias("FTHG"),
        avg(silver_table.HS).alias("HSH"),
        avg(silver_table.HST).alias("HSTH"),
        avg(silver_table.HC).alias("HCH"),
        sum(silver_table.HS).alias("SHS"),
        sum(silver_table.FTHG).alias("SFTHG")
    
    ) \
    .withColumnRenamed("HomeTeam", "Team") 

homeTeam_table.show(n=30)
homeTeam_table.count()

+--------------+----+-------+------------------+------------------+------------------+------------------+---+-----+
|          Team|Date|    Div|              FTHG|               HSH|              HSTH|               HCH|SHS|SFTHG|
+--------------+----+-------+------------------+------------------+------------------+------------------+---+-----+
|           QPR|2014|England|               1.7|              17.2|               5.2|               6.6|172|   17|
|      Cagliari|2016|  Italy|2.2222222222222223| 16.77777777777778| 5.555555555555555| 6.666666666666667|151|   20|
|    Leverkusen|2014|Germany|               2.0| 6.777777777777778| 2.888888888888889|3.5555555555555554| 61|   18|
|    Man United|2016|England|              1.65|              17.0|              6.05|              7.45|340|   33|
|     Newcastle|2019|England|1.8888888888888888| 15.88888888888889| 5.666666666666667| 5.333333333333333|143|   17|
|Crystal Palace|2014|England|1.1111111111111112|              12.0| 3.88

380

In [8]:
awayTeam_table = silver_table \
    .groupBy("AwayTeam", "Date", "Div") \
    .agg(
        avg(silver_table.FTAG).alias("FTAG"),
        avg(silver_table.AS).alias("ASA"),
        avg(silver_table.AST).alias("ASTA"),
        avg(silver_table.AC).alias("ACA"),
        sum(silver_table.AS).alias("SAS"),
        sum(silver_table.FTAG).alias("SFTAG")
    ) \
    .withColumnRenamed("AwayTeam", "Team") 

awayTeam_table.show(n=30)
awayTeam_table.count()

+--------------+----+-------+------------------+------------------+------------------+------------------+---+-----+
|          Team|Date|    Div|              FTAG|               ASA|              ASTA|               ACA|SAS|SFTAG|
+--------------+----+-------+------------------+------------------+------------------+------------------+---+-----+
|           QPR|2014|England|0.4444444444444444|10.666666666666666|2.2222222222222223|               4.0| 96|    4|
|      Cagliari|2016|  Italy|0.7777777777777778| 6.666666666666667| 15.11111111111111|2.3333333333333335| 60|    7|
|    Leverkusen|2014|Germany|              1.25|              4.75|            16.125|             2.125| 38|   10|
|    Man United|2016|England|1.2777777777777777|11.444444444444445|3.7222222222222223| 4.888888888888889|206|   23|
|Crystal Palace|2014|England|               1.0|               9.4|               3.4|               5.0| 94|   10|
|     Newcastle|2019|England|1.1111111111111112| 10.11111111111111| 2.88

380

In [9]:
#join of the two dataFrames
join_table = homeTeam_table.join(awayTeam_table,['Team', 'Date', 'Div'], how='left') \
    .withColumnRenamed("Team", "Equipa") \
    .withColumnRenamed("Div", "Liga") \
    .withColumnRenamed("Date", "Data") \
    .orderBy("Date", "Liga") \
    .withColumn("Media_golos", round(((col("FTAG") + col("FTHG"))/2), 2)) \
    .withColumn("Media_remates", round(((col("HSH") + col("ASA"))/2), 2)) \
    .withColumn("Media_remates_baliza", round(((col("HSTH") + col("ASTA"))/2), 2)) \
    .withColumn("Media_cantos", round(((col("HCH") + col("ACA"))/2), 2)) \
    .withColumn("Eficacia", round(((col("SFTHG") + col("SFTAG")) / (col("SHS") + col("SAS")) * 100), 2)) \
    .select("Liga", "Equipa", "Data", "Media_golos", "Media_remates", "Media_remates_baliza", "Media_cantos", "Eficacia")


#WRITE TO DELTA TABLE
join_table \
    .write \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/demo/gold/UC3_LS.db/stats/")



join_table.show()
join_table.count()

+-------+--------------+----+-----------+-------------+--------------------+------------+--------+
|   Liga|        Equipa|Data|Media_golos|Media_remates|Media_remates_baliza|Media_cantos|Eficacia|
+-------+--------------+----+-----------+-------------+--------------------+------------+--------+
|England|       Chelsea|2014|       2.17|        16.34|                6.07|        6.38|   13.31|
|England|       Arsenal|2014|        1.8|        16.93|                5.77|        7.83|   10.66|
|England|       Swansea|2014|       1.25|        10.68|                3.53|        3.34|   11.76|
|England|     Newcastle|2014|       1.17|        12.57|                3.88|        6.07|    9.32|
|England|     Leicester|2014|       0.92|        11.25|                3.41|        5.48|    8.06|
|England|   Aston Villa|2014|       0.59|         9.91|                2.83|        3.53|    5.85|
|England|      Man City|2014|       2.14|        16.74|                5.58|        6.78|   12.89|
|England| 

380

In [10]:
# check the results in the table
spark.table("UC3_LS.stats").toPandas()

,Liga,Equipa,Data,Media_golos,Media_remates,Media_remates_baliza,Media_cantos,Eficacia
0,England,Man United,2016,1.46,14.22,4.89,6.17,10.26
1,England,Hull,2016,0.83,10.12,3.48,4.26,8.33
2,England,Stoke,2016,1.13,11.13,3.50,4.74,10.17
3,England,Man City,2016,1.92,15.77,5.09,6.79,12.27
4,England,Leicester,2016,1.43,12.61,4.03,5.51,11.46
...,...,...,...,...,...,...,...,...
375,Italy,Inter,2019,1.39,5.39,9.35,3.19,25.00
376,Italy,Roma,2019,1.67,6.45,7.95,3.44,25.81
377,Italy,Udinese,2019,1.23,8.63,8.82,3.59,14.20
378,Italy,Chievo,2019,0.58,8.93,7.95,3.26,6.51


In [11]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta. `hdfs://hdfs-nn:9000/demo/gold/UC3_LS.db/stats/`
""").show()

++
||
++
++



In [4]:
spark.sql("""
DROP TABLE IF EXISTS UC3_LS.stats_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS UC3_LS.stats_presto (
        Liga STRING,
        Equipa STRING,
        Data STRING,
        Media_golos DOUBLE,
        Media_remates DOUBLE,
        Media_remates_baliza DOUBLE,
        Media_cantos DOUBLE,
        Eficacia DOUBLE
        )
        
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/demo/gold/UC3_LS.db/stats/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++

